# Training an XOR network using PySyft

In this notebook we'll be training an XOR network using PySyft. 
What's special about this is that the training data will be divided between two domain nodes that are owned by different parties.

In [1]:
import syft as sy
import numpy as np
from syft.core.adp.entity import Entity
from syft.core.tensor.smpc.mpc_tensor import MPCTensor
sy.logger.remove()

## Party 1 - OpenMined's team in the UK

In [15]:
# Party 1 logs into their domain node
p1_domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into adp... done!


In [27]:
# They take their training data for the XOR network
training_data_p1 = np.array(
    [[0,0,1],
    [0,1,1]], dtype=np.int64
)

# rasswanth = Entity(name="Rasswanth")

training_data_p1 = sy.Tensor(training_data_p1).private(
    min_val=0,
    max_val=1,
    entities=["Rasswanth"] * training_data_p1.shape[0],
    ndept=True
)

training_targets_p1 = np.array([[0,1]]).T

training_targets_p1 = sy.Tensor(training_targets_p1).private(
    min_val=0,
    max_val=1,
    entities=["Rasswanth"] * training_data_p1.shape[0],
    ndept=True
)

p1_domain_node.load_dataset(
    assets={
        "training_data":training_data_p1,
        "training_targets":training_targets_p1
    },
    name="Our training data for XOR networks!",
    description="Collected on Jan 27 2022"
)

# This is just used to check our results afterwards.
# t1 = sy.Tensor(np.array([1,2,3])).send(p1)

<class 'syft.core.tensor.tensor.Tensor'>
<class 'syft.core.tensor.autodp.ndim_entity_phi.NDimEntityPhiTensor'>
Dataset is uploaded successfully !!! 🎉                                                                                                                                       

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


<hr>
Let's check and make sure our dataset  was uploaded!

In [33]:
p1_domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> int64",0731cbfb-818e-4980-8236-6238676bbad7
[1],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",fb3d41c9-1c39-4544-9501-280b6997ff8d
[2],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",de26285c-506b-488e-bfa4-0cd9d62c64bc


Looks good!

Now the last thing this team needs to do is to create an account for the Data Scientist who will be training the network using their data.

In [34]:
p1_domain_node.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "budget": 100
    }
)

In [35]:
p1_domain_node.users

,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,c5c604b6de386b3be3c6705a955a6f4591eddf9465d13d...,Owner,None,None,None,NaN,2022-04-20 23:46:24.761424,5.55
1,2,sheldon@caltech.edu,Sheldon Cooper,100.00,a324011a6b854de50de72830bc3fc6c533f7f1ff0cd07a...,Data Scientist,Jane Doe,,,1.0,2022-04-21 03:03:25.055619,100.00


<hr>

## Party 2 - Facebook's team in Menlo Park

In [31]:
DOMAIN2_PORT = 8082
p2_domain_node = sy.login(email="info@openmined.org",password="changethis",port=DOMAIN2_PORT)
training_data_p2 = np.array([
    [1,0,1],
    [1,1,1]]
)

training_data_p2 = sy.Tensor(training_data_p2).private(
    min_val=0,
    max_val=1,
    entities=["Rasswanth"] * training_data_p2.shape[0],
    ndept=True
)


training_targets_p2 = np.array([[1,0]]).T
training_targets_p2 = sy.Tensor(training_targets_p2).private(
    min_val=0,
    max_val=1,
    entities=["Rasswanth", "Rasswanth"],
    ndept=True
)

p2_domain_node.load_dataset(
    assets={
        "training_data":training_data_p2,
        "training_targets":training_targets_p2
    },
    name="Our training data for XOR networks!",
    description="Collected on Jan 27 2022"
)

# Used to check our results afterwards
# t2 = sy.Tensor(np.array([1,2,3])).send(p2)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into musing_sutskever... done!
Dataset is uploaded successfully !!! 🎉                                                                                                                                       

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [32]:
p2_domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Italy Trade Data - First few rows,A collection of reports from iStat's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Italy Trade""] -> int64",4514b06f-940f-44a5-b337-8fe9ee3ce405
[1],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",e73b2c57-7f2c-4ce8-bff2-c669c42cde1e


Looks good!

Now the last thing this team needs to do is to create an account for the Data Scientist who will be training the network using their data.

In [36]:
p2_domain_node.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "budget": 100
    }
)

In [37]:
p2_domain_node.users

,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,cb642c15f0fe6ac49184e69f5b50cf03d7efc3accf6c17...,Owner,None,None,None,NaN,2022-04-21 00:16:40.288027,5.55
1,2,sheldon@caltech.edu,Sheldon Cooper,100.00,6ebc6514de733c3d97ca872e9c2ca01b3530612dd94fee...,Data Scientist,Jane Doe,,,1.0,2022-04-21 03:04:10.950905,100.00


<hr>
<hr>

### Data Scientists

In [38]:
ds_domain1 = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
ds_domain1.datasets

Connecting to localhost... done! 	 Logging into adp... done!


Idx,Name,Description,Assets,Id
[0],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> int64",0731cbfb-818e-4980-8236-6238676bbad7
[1],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",fb3d41c9-1c39-4544-9501-280b6997ff8d
[2],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",de26285c-506b-488e-bfa4-0cd9d62c64bc


In [39]:
ds_domain2 = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)
ds_domain2.datasets

Connecting to localhost... done! 	 Logging into musing_sutskever... done!


Idx,Name,Description,Assets,Id
[0],Italy Trade Data - First few rows,A collection of reports from iStat's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Italy Trade""] -> int64",4514b06f-940f-44a5-b337-8fe9ee3ce405
[1],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",e73b2c57-7f2c-4ce8-bff2-c669c42cde1e


In [40]:
ds_domain1.privacy_budget

100.0

In [41]:
ds_domain2.privacy_budget

100.0

In [8]:
X, y = sy.concatenate(p1, p2, X, y)

## This replaces the following code:
# d1 = X.send(p1)
# X = MPCTensor(secret=d1,parties=parties,shape=X.shape)
# d2 = y.send(p1)
# y = MPCTensor(secret=d2,parties=parties,shape=y.shape)

In [4]:
def relu(x,deriv=False):
    if deriv==True:
        return x>0
    return x*(x>0)

In [5]:
layer0_weights = 2*np.random.random((3,4)) - 1
layer1_weights = 2*np.random.random((4,1)) - 1

In [11]:
for j in range(1):
    # Forward propagation
    layer1_inputs = relu(X @ layer0_weights)  ; layer1_inputs.block
    layer2_inputs = relu(layer1_inputs @ layer1_weights) ; layer2_inputs.block 
    
    # Calculate errors
    layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True) ; layer2_inputs_delta.block
    layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True) ; layer1_inputs_delta.block
    
    # Update weights
    layer1_weights  = layer1_weights + layer1_inputs.T @ layer2_inputs_delta ; layer1_weights.block
    layer0_weights =  layer0_weights + X.T @ layer1_inputs_delta ; layer0_weights.block

Computing...:   0%|                                                                                                                                                    | 0/64 [00:00<?, ?it/s]/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:72: RuntimeWarning: overflow encountered in multiply
  res = op(np.empty(x_shape), np.empty(y_shape)).shape
Computing...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:52<00:00,  1.21it/s]


KeyboardInterrupt: 

Let's see how our privacy budget changed as a result of training for a single epoch:

In [40]:
ds_domain1.privacy_budget

100.0

In [41]:
ds_domain2.privacy_budget

100.0

And voila! We've trained a neural network using PySyft's adversaril differential privacy system and its secure multiparty computation system working in tandem.


<hr>
<hr>
Demo ends above- the cells below are for temporary Debugging:

In [6]:
X



MPCTensor.shape=(4, 3)
	 .child[0] = <TensorPointer -> adp:392155ad45bb4a50a1adbb9430ee2fb9>
	 .child[1] = <TensorPointer -> loving_larochelle:4ad8c4939bf24f49a284719fc32846aa>

In [7]:
layer0_weights

array([[0.11840966, 0.76486982, 0.19141629, 0.55507458],
       [0.91480034, 0.99278924, 0.33589298, 0.76190623],
       [0.43318126, 0.46445256, 0.24577587, 0.139101  ]])

In [9]:
layer1_inputs = relu(X @ layer0_weights)  ; layer1_inputs.block

Computing...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [01:07<00:00,  1.06s/it]


KeyboardInterrupt: 

In [7]:
layer2_inputs = relu(layer1_inputs @ layer1_weights) ; layer2_inputs.block 

/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:72: RuntimeWarning: overflow encountered in matmul
  res = op(np.empty(x_shape), np.empty(y_shape)).shape
Computing...: 100%|██████████| 64/64 [00:38<00:00,  1.65it/s]




MPCTensor.shape=(4, 1)
	 .child[0] = <TensorPointer -> canada:a4369566d34fddbe9eb9ab447061bb85>
	 .child[1] = <TensorPointer -> italy:a4369566d34fddbe9eb9ab447061bb85>

In [8]:
layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True) ; layer2_inputs_delta.block

Computing...: 100%|██████████| 64/64 [00:38<00:00,  1.65it/s]




MPCTensor.shape=(4, 1)
	 .child[0] = <TensorPointer -> canada:b7d501bf26ff7cc1581fce1478a9ba5b>
	 .child[1] = <TensorPointer -> italy:b7d501bf26ff7cc1581fce1478a9ba5b>

In [11]:
layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True) ; layer1_inputs_delta.block

Computing...: 100%|██████████| 64/64 [00:38<00:00,  1.65it/s]




MPCTensor.shape=(4, 4)
	 .child[0] = <TensorPointer -> canada:c2a70790ad2c8968289539821840e5a3>
	 .child[1] = <TensorPointer -> italy:c2a70790ad2c8968289539821840e5a3>

In [ ]:
layer1_weights  = layer1_inputs.T @ layer2_inputs_delta  + layer1_weights; layer1_weights.block

In [ ]:
layer0_weights =  X.T @ layer1_inputs_delta  + layer0_weights; layer0_weights.block

In [26]:
res = X.T @ layer1_inputs_delta

In [33]:
v1 = X.T
v2 = layer1_inputs_delta

In [34]:
def get_val(val):
    t1 = val.child[0].get_copy()
    t2 = val.child[0].get_copy()
    return t1+t2

In [35]:
v1 = get_val(v1)
v2 = get_val(v2)

In [51]:
v1.child.min_vals.to_numpy() @ v2.child.min_vals.to_numpy()

array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]])

In [50]:
v2.child.min_vals

<lazyrepeatarray data: [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]] -> shape: (4, 4)>

In [ ]:
layer0_weights_dp = layer0_weights.publish(sigma=1e4)
layer1_weights_dp = layer1_weights.publish(sigma=1e4)

In [ ]:
print(layer0_weights_dp.get_copy())
print(layer1_weights_dp.get_copy())